In [28]:
from langchain.agents import initialize_agent, AgentType
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [29]:
# load environment variables 
import os
from dotenv import dotenv_values, load_dotenv
load_dotenv()
openai_api_key = os.environ['OPENAI_API_KEY']

In [30]:
model = "gpt-3.5-turbo"
temperature = 0
llm = ChatOpenAI(model=model, temperature=temperature, openai_api_key=openai_api_key)

In [31]:
from triage import triage_biopsy
triage_biopsy(1.5, True, 1.2, False, 0.5, True)

(Core(n='first', length=1.5, cortex=True),
 'Assign the first core as the diagnostic core. If the amount of cortext is less than 0.5 cm then please the entire core in formalin for light microscopy. Otherwise, divide diagnostic core in three parts with a minimum of 0.1-0.2 cm of cortex fixed in 2.5% glutaraldehyde for EM, 0.3 cm cortex frozen in OCT for IF, and the remaining amount of tissue fixed in 10% formalin for LM. \nThe second biopsy core should be embedded in OCT, frozen on dry ice and shipped to the central biorepository where it will be distributed to the TISs.')

In [5]:
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class TriageInput(BaseModel):
    core1_length: float = Field()
    core1_cortex: bool = Field()
    core2_length: float = Field()
    core2_cortex: bool = Field()
    core3_length: float = Field()
    core3_cortex: bool = Field()
    n_passes: int = Field()

In [33]:
tools = [
    StructuredTool(
        name = "triage_biopsy",
        func = triage_biopsy,
        description = "Use this tool to triage a kidney biopsy. It produces instructions to process the biopsy bases on the length of the cores and whether renal or kidney cortext is visualized in them. If cortex is not mentioned in the text, then assume that it was not visualized",
        args_schema = TriageInput
    )
]

In [26]:
# AgentType = STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION | OPENAI_FUNCTIONS | OPENAI_MULTI_FUNCTIONS
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [27]:
agent.run("You obtain three cores, first core is 1.5 cm in length, second is 1 cm, and thrid is 0.5 cm. You see cortex in first core. How would you triage the tissue?")



> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.5, 'core1_cortex': True, 'core2_length': 1, 'core2_cortex': False, 'core3_length': 0.5, 'core3_cortex': False, 'n_passes': 3}`


(Core(n='first', length=1.5, cortex=True), 'Assign the first core as the diagnostic core. If the amount of cortext is less than 0.5 cm then please the entire core in formalin for light microscopy. Otherwise, divide diagnostic core in three parts with a minimum of 0.1-0.2 cm of cortex fixed in 2.5% glutaraldehyde for EM, 0.3 cm cortex frozen in OCT for IF, and the remaining amount of tissue fixed in 10% formalin for LM. \n')Based on the information provided, here is how you should triage the tissue:

1. Assign the first core (1.5 cm in length) as the diagnostic core.
2. If the amount of cortex in the first core is less than 0.5 cm, please place the entire core in formalin for light microscopy.
3. If the amount of cortex in the first core is 0.5 cm or more, divide the di

'Based on the information provided, here is how you should triage the tissue:\n\n1. Assign the first core (1.5 cm in length) as the diagnostic core.\n2. If the amount of cortex in the first core is less than 0.5 cm, please place the entire core in formalin for light microscopy.\n3. If the amount of cortex in the first core is 0.5 cm or more, divide the diagnostic core into three parts:\n   - Take a minimum of 0.1-0.2 cm of cortex and fix it in 2.5% glutaraldehyde for electron microscopy.\n   - Take 0.3 cm of cortex and freeze it in OCT for immunofluorescence.\n   - Fix the remaining amount of tissue in 10% formalin for light microscopy.\n\nPlease follow these instructions to process the biopsy.'

In [35]:
agent.run("You obtain three cores, first core is 1.5 cm in length, second is 1 cm, and third is 0.5 cm. Cortex was not visualized. How would you triage the tissue?")



> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.5, 'core1_cortex': False, 'core2_length': 1, 'core2_cortex': False, 'core3_length': 0.5, 'core3_cortex': False, 'n_passes': 3}`


(Core(n='first', length=1.5, cortex=False), 'Assign the first core as the diagnostic core. A minimum of 0.1-0.2 cm of tissue from both tips of the diagnostic core should be placed in  glutaraldehyde for EM, a 0.3 cm of tissue from both tips will be frozen in OCT for IF, and the remaining tissue will be placed in formalin for LM. \nThe remaining two cores will be designated for tissue interrogation and triaged/processed as cores #2 and #3.')Based on the information provided, I would triage the tissue as follows:

1. Assign the first core (1.5 cm in length) as the diagnostic core.
   - A minimum of 0.1-0.2 cm of tissue from both tips of the diagnostic core should be placed in glutaraldehyde for electron microscopy (EM).
   - A 0.3 cm of tissue from both tips will be fro

'Based on the information provided, I would triage the tissue as follows:\n\n1. Assign the first core (1.5 cm in length) as the diagnostic core.\n   - A minimum of 0.1-0.2 cm of tissue from both tips of the diagnostic core should be placed in glutaraldehyde for electron microscopy (EM).\n   - A 0.3 cm of tissue from both tips will be frozen in OCT for immunofluorescence (IF).\n   - The remaining tissue will be placed in formalin for light microscopy (LM).\n\n2. The remaining two cores will be designated for tissue interrogation and triaged/processed as cores #2 and #3.\n\nPlease note that this is a general recommendation and may vary depending on specific protocols or guidelines followed by your institution.'